In [1]:
import pandas as pd
import numpy as np
import re
import spacy
import transformers

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

C:\Users\krist\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Summarize one text

def summarize(text):
    
    # Removing the first common part

    splitted_text = text.split('Numac')
    splitted_text = splitted_text[1]

    # removing symbols

    no_symbols = splitted_text.replace('\n',' ')
    no_symbols = no_symbols.replace('/','')
    no_symbols = no_symbols.replace('§','')
    
    # Removing 'begin, eerste, laatste,...' from the end of the text and creating our main text.
    index = no_symbols.rfind('begin')
    
    corpus = no_symbols[:index]
    
    # Starting the summarizer
    
    import transformers
    undisputed_best_model = transformers.MBartForConditionalGeneration.from_pretrained(
        "ml6team/mbart-large-cc25-cnn-dailymail-nl-finetune")
    
    tokenizer = transformers.MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")
    summarization_pipeline = transformers.pipeline(
        task="summarization",
        model=undisputed_best_model,
        tokenizer=tokenizer,
    )
    summarization_pipeline.model.config.decoder_start_token_id = tokenizer.lang_code_to_id[
        "nl_XX"
    ]


    article = corpus
    summary = summarization_pipeline(
        article,
        do_sample=True,
        top_p=0.75,
        top_k=50,
        # num_beams=4,
        min_length=50,
        early_stopping=True,
        truncation=True,
    )[0]["summary_text"]
    
    return summary

In [3]:
def scrape(url):
    
    # Decide on the webdriver to use.

    driver = webdriver.Edge()
    
    # URL compatibility check
    
    substring = 'body'
    
    if substring not in url:
        url = url.replace('article','article_body')
    print(url)
    
    # Go to the website

    driver.get(url)
    
    # Scrape the text from the website
    for elem in driver.find_elements(By.TAG_NAME,"body"):
        text = elem.text
         
    return text

In [4]:
url = 'https://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&caller=summary&pub_date=2022-06-13&numac=2021205279'

In [5]:
text = scrape(url)

https://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&caller=summary&pub_date=2022-06-13&numac=2021205279


In [14]:
summarize(text)

'Het Grondwettelijk Hof in Brussel heeft een beroep ingediend tegen de ZIV-wet, die de besparingsmaatregelen bij de farmaceutische specialiteiten in het kader van de gezondheidszorgbegroting 2020 moet vernietigen.'

In [ ]:
    # Creating Tokens

    nlp = spacy.load('nl_core_news_sm')
    text = no_symbols
    tokenized = nlp(text)
    tokens = [token.text for token in tokenized]
    
    # Removing stopwords
    stopwords = spacy.lang.nl.stop_words.STOP_WORDS
    
    # Lemmatizing the text
    text_no_stop = [lemma for lemma in tokens if lemma not in stopwords]
    cleaned = ' '.join(text_no_stop)